In [15]:
import geopandas as gpd
import laspy
import pdal
import json,os
import pandas as pd

In [2]:
json_to_store = {
    "pipeline": [
        {
"bounds": "([-10425171.940, -10423171.940], [5164494.710, 5166494.710])",
"filename": "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json",
"type": "readers.ept",
"tag": "readdata"
        },
        {
            "limits": "Classification![7:7]",
            "type": "filters.range",
            "tag": "nonoise"
        },
        {
            "assignment": "Classification[:]=0",
            "tag": "wipeclasses",
            "type": "filters.assign"
        },
        {
            "out_srs": "EPSG:26915",
            "tag": "reprojectUTM",
            "type": "filters.reprojection"
        },
        {
            "tag": "groundify",
            "type": "filters.smrf"
        },
        {
            "limits": "Classification[2:2]",
            "type": "filters.range",
            "tag": "classify"
        },
        {
            "filename": "../data/iowa.laz",
            "inputs": [ "classify" ],
            "tag": "writerslas",
            "type": "writers.las"
        },
        {
            "filename": "../data/iowa.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "inputs": [ "writerslas" ],
            "nodata": -9999,
            "output_type": "idw",
            "resolution": 1,
            "type": "writers.gdal",
            "window_size": 6
        }
    ]
}

In [26]:
json_object = json.dumps(json_to_store, indent = 4)
  
# Writing to sample.json
with open("../data/iowa.json", "w") as outfile:
    outfile.write(json_object)

In [30]:
json_to_store['pipeline']

[{'bounds': '([-10425171.940, -10423171.940], [5164494.710, 5166494.710])',
  'filename': 'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json',
  'type': 'readers.ept',
  'tag': 'readdata'},
 {'limits': 'Classification![7:7]', 'type': 'filters.range', 'tag': 'nonoise'},
 {'assignment': 'Classification[:]=0',
  'tag': 'wipeclasses',
  'type': 'filters.assign'},
 {'out_srs': 'EPSG:26915',
  'tag': 'reprojectUTM',
  'type': 'filters.reprojection'},
 {'tag': 'groundify', 'type': 'filters.smrf'},
 {'limits': 'Classification[2:2]', 'type': 'filters.range', 'tag': 'classify'},
 {'filename': '../data/iowa.laz',
  'inputs': ['classify'],
  'tag': 'writerslas',
  'type': 'writers.las'},
 {'filename': '../data/iowa.tif',
  'gdalopts': 'tiled=yes,     compress=deflate',
  'inputs': ['writerslas'],
  'nodata': -9999,
  'output_type': 'idw',
  'resolution': 1,
  'type': 'writers.gdal',
  'window_size': 6}]

In [4]:
os.system('pdal pipeline ../data/iowa.json --debug')

(PDAL Debug) Debugging...
(pdal pipeline readers.ept Debug) Query bounds: ([-10425171.94, -10423171.94], [5164494.71, 5166494.71], [-1.797693134862316e+308, 1.797693134862316e+308])
Threads: 15
(pdal pipeline Debug) Executing pipeline in standard mode.
(pdal pipeline filters.smrf Debug) progressiveFilter: radius = 1	2234705 ground cells	3302 non-ground cells	(0.15% of cells contain ground)
(pdal pipeline filters.smrf Debug) progressiveFilter: radius = 1	2026879 ground cells	211128 non-ground cells	(9.43% of cells contain ground)
(pdal pipeline filters.smrf Debug) progressiveFilter: radius = 2	1979134 ground cells	258873 non-ground cells	(11.57% of cells contain ground)
(pdal pipeline filters.smrf Debug) progressiveFilter: radius = 3	1959708 ground cells	278299 non-ground cells	(12.44% of cells contain ground)
(pdal pipeline filters.smrf Debug) progressiveFilter: radius = 4	1948026 ground cells	289981 non-ground cells	(12.96% of cells contain ground)
(pdal pipeline filters.smrf Debug) p

0

In [5]:
os.system('gdaldem color-relief ../data/iowa.tif ../data/dem-colors.txt ../data/iowa-color.png')

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [6]:
pipeline=pdal.Reader('../data/iowa.laz').pipeline()

In [7]:
count = pipeline.execute()

In [8]:
arrays = pipeline.arrays
arrays

[array([(447498.6 , 4654050.85, 273.11, 167, 1, 1, 0, 0, 2, 5., 70, 79, 55121.22966839, 0, 0, 0),
        (447496.79, 4654050.48, 273.13, 148, 1, 1, 0, 0, 2, 5., 70, 79, 55121.22967856, 0, 0, 0),
        (447490.44, 4654050.26, 273.15, 133, 1, 1, 1, 0, 2, 5., 70, 79, 55121.24323788, 0, 0, 0),
        ...,
        (446338.1 , 4652585.34, 282.08,  90, 1, 1, 1, 0, 2, 2., 69, 85, 56180.39031803, 0, 0, 0),
        (446271.66, 4652586.13, 282.19,  12, 1, 1, 1, 0, 2, 4., 69, 85, 56180.5284318 , 0, 0, 0),
        (446206.28, 4652586.53, 282.78,  44, 1, 1, 1, 0, 2, 6., 69, 85, 56180.66654557, 0, 0, 0)],
       dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])]

In [9]:
metadata = pipeline.metadata

In [10]:
log = pipeline.log

In [11]:
array = arrays.pop()

In [12]:
array.shape

(1399943,)

In [37]:
def load_geolidar(csv,filename=None,path_to_json=None):
    # Open the csv
    _csv = pd.read_csv(csv)
    # Open the JSON
    with open(path_to_json) as json_file:
        the_json = json.load(json_file)
    # Open an empty csv on S3
    with open(filename, 'w') as fp:
        fp.write('id,x,y,z\n')
    # write into csv
    print(the_json)
    for _id, _filename, _bounds, _polygon in zip(_csv.id, _csv.filename, _csv.bounds, _csv.polygon):
        the_json[0]['filename'] = _filename
        the_json[0]['bounds'] = _bounds
        the_json[4]['polygon'] = _polygon
        pipeline = pdal.Pipeline(json.dumps(the_json))
        try:
            pipeline.execute()
            xyz = pipeline.arrays[0][['X','Y','Z']][0]
            fp.write(f'{_id},{xyz[0]:.2f},{xyz[1]:.2f},{xyz[2]:.2f}\n')
        except RuntimeError as e:
                print(e)
                # RuntimeError: filters.hag: Input PointView does not have any points classified as ground
                print('RunTime Error, writing 0s and moving to next bounds')
                fp.write(f'{_id},0,0,0\n')
                pass

In [38]:
load_geolidar('../data/iowa_template.csv','../data/iowa.csv','../data/iowa.json')

{'pipeline': [{'bounds': '([-10425171.940, -10423171.940], [5164494.710, 5166494.710])', 'filename': 'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json', 'type': 'readers.ept', 'tag': 'readdata'}, {'limits': 'Classification![7:7]', 'type': 'filters.range', 'tag': 'nonoise'}, {'assignment': 'Classification[:]=0', 'tag': 'wipeclasses', 'type': 'filters.assign'}, {'out_srs': 'EPSG:26915', 'tag': 'reprojectUTM', 'type': 'filters.reprojection'}, {'tag': 'groundify', 'type': 'filters.smrf'}, {'limits': 'Classification[2:2]', 'type': 'filters.range', 'tag': 'classify'}, {'filename': '../data/iowa.laz', 'inputs': ['classify'], 'tag': 'writerslas', 'type': 'writers.las'}, {'filename': '../data/iowa.tif', 'gdalopts': 'tiled=yes,     compress=deflate', 'inputs': ['writerslas'], 'nodata': -9999, 'output_type': 'idw', 'resolution': 1, 'type': 'writers.gdal', 'window_size': 6}]}


In [40]:
with open('../data/iowa.json') as json_file:
        the_json = json.load(json_file)